### 유저 정보 가져오기

In [50]:
import requests
import json
import pandas as pd
import time
from urllib import parse # 한글

import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pymysql
import numpy as np
from einops import rearrange, einsum

import torch
import mysql

In [2]:
api_key =  'RGAPI-79190ab9-eb9b-4abb-a486-cb694c20cc07' # 본인의 API KEY 입력
REQUEST_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

In [111]:
user_url = f'https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page={1}&api_key={api_key}'
url_now = user_url.format(api_key)
reqq =  requests.get(url_now, REQUEST_HEADERS).text
df = json.loads(reqq)
df = pd.DataFrame(df)
df = df.drop(['leagueId', 'queueType', 'rank', 'inactive', 'freshBlood', 'hotStreak'], axis = 1)

puuids = []

for summonerId in df['summonerId']:
    user_puuids_url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}"
    response = requests.get(user_puuids_url, headers=REQUEST_HEADERS)
    puuid = response.json().get('puuid')
    if puuid:  
        puuids.append(puuid)
        break
    else:
        continue

match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuids[0]}/ids?start={1}&count={5}"
match_ids = requests.get(match_url, headers=REQUEST_HEADERS).json()



In [115]:
match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_ids[3]}?api_key={api_key}"
response = requests.get(match_url, headers=REQUEST_HEADERS).json()

In [125]:
feature_list = response['info']['participants'][0]['item2']
feature_list

3076

In [122]:
feature_list = response['info']['participants']
lane_dict = {}
for i in range(10):
    print(feature_list[i]['teamPosition'])


TOP
JUNGLE
MIDDLE
BOTTOM
UTILITY
TOP
JUNGLE
MIDDLE
BOTTOM
UTILITY


In [117]:
lane_dict

{0: ('TOP', 'SOLO'),
 1: ('JUNGLE', 'NONE'),
 2: ('MIDDLE', 'SOLO'),
 3: ('BOTTOM', 'CARRY'),
 4: ('BOTTOM', 'SUPPORT'),
 5: ('TOP', 'SOLO'),
 6: ('MIDDLE', 'SUPPORT'),
 7: ('MIDDLE', 'CARRY'),
 8: ('JUNGLE', 'NONE'),
 9: ('BOTTOM', 'SOLO')}

In [49]:
response = requests.get('https://ddragon.leagueoflegends.com/cdn/15.3.1/data/en_US/item.json', headers=REQUEST_HEADERS).json()

item_mapping = {item_id: info["name"] for item_id, info in response["data"].items()}
item_mapping
# # 특정 아이템 ID로 이름 찾기 (예: 1001)
# item_id = "1001"
# print(item_mapping.get(item_id, "Unknown Item"))


{'1001': 'Boots',
 '1004': 'Faerie Charm',
 '1006': 'Rejuvenation Bead',
 '1011': "Giant's Belt",
 '1018': 'Cloak of Agility',
 '1026': 'Blasting Wand',
 '1027': 'Sapphire Crystal',
 '1028': 'Ruby Crystal',
 '1029': 'Cloth Armor',
 '1031': 'Chain Vest',
 '1033': 'Null-Magic Mantle',
 '1035': 'Emberknife',
 '1036': 'Long Sword',
 '1037': 'Pickaxe',
 '1038': 'B. F. Sword',
 '1039': 'Hailblade',
 '1040': 'Obsidian Edge',
 '1042': 'Dagger',
 '1043': 'Recurve Bow',
 '1052': 'Amplifying Tome',
 '1053': 'Vampiric Scepter',
 '1054': "Doran's Shield",
 '1055': "Doran's Blade",
 '1056': "Doran's Ring",
 '1057': 'Negatron Cloak',
 '1058': 'Needlessly Large Rod',
 '1082': 'Dark Seal',
 '1083': 'Cull',
 '1101': 'Scorchclaw Pup',
 '1102': 'Gustwalker Hatchling',
 '1103': 'Mosstomper Seedling',
 '1104': 'Eye of the Herald',
 '1105': 'Mosstomper Seedling',
 '1106': 'Gustwalker Hatchling',
 '1107': 'Scorchclaw Pup',
 '126697': 'Hubris<br>',
 '1500': 'Penetrating Bullets',
 '1501': 'Fortification',
 '15

In [53]:
import os

# 챔피언 리스트 저장
item = []

# JSON 파일 가져오기
datas = requests.get('https://ddragon.leagueoflegends.com/cdn/15.3.1/data/en_US/item.json')
datas = datas.json()

for data in datas["data"]:
    item.append(data)

# 저장할 폴더 경로 설정
save_folder = r"C:\Users\JiWoongChoi\pythonproject\riotproject\riotproject\static\images\item"

# 폴더 없으면 생성
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

for it in item:
    
    save_path = os.path.join(save_folder, it + ".png")

    if os.path.exists(save_path):
        print(f"⏩ {it}.png 이미 존재하여 건너뜀")
        continue

    image_url = f"https://ddragon.leagueoflegends.com/cdn/15.2.1/img/item/{it}.png"

    download_image = requests.get(image_url)

    if download_image.status_code == 200:  # 다운로드 성공 여부 확인
        with open(save_path, 'wb') as file:
            file.write(download_image.content)
        print(f"✅ {it}.png 다운로드 완료")
    else:
        print(f"❌ {it}.png 다운로드 실패 (HTTP {download_image.status_code})")

✅ 1001.png 다운로드 완료
✅ 1004.png 다운로드 완료
✅ 1006.png 다운로드 완료
✅ 1011.png 다운로드 완료
✅ 1018.png 다운로드 완료
✅ 1026.png 다운로드 완료
✅ 1027.png 다운로드 완료
✅ 1028.png 다운로드 완료
✅ 1029.png 다운로드 완료
✅ 1031.png 다운로드 완료
✅ 1033.png 다운로드 완료
✅ 1035.png 다운로드 완료
✅ 1036.png 다운로드 완료
✅ 1037.png 다운로드 완료
✅ 1038.png 다운로드 완료
✅ 1039.png 다운로드 완료
✅ 1040.png 다운로드 완료
✅ 1042.png 다운로드 완료
✅ 1043.png 다운로드 완료
✅ 1052.png 다운로드 완료
✅ 1053.png 다운로드 완료
✅ 1054.png 다운로드 완료
✅ 1055.png 다운로드 완료
✅ 1056.png 다운로드 완료
✅ 1057.png 다운로드 완료
✅ 1058.png 다운로드 완료
✅ 1082.png 다운로드 완료
✅ 1083.png 다운로드 완료
✅ 1101.png 다운로드 완료
✅ 1102.png 다운로드 완료
✅ 1103.png 다운로드 완료
✅ 1104.png 다운로드 완료
✅ 1105.png 다운로드 완료
✅ 1106.png 다운로드 완료
✅ 1107.png 다운로드 완료
✅ 126697.png 다운로드 완료
✅ 1500.png 다운로드 완료
✅ 1501.png 다운로드 완료
✅ 1502.png 다운로드 완료
✅ 1503.png 다운로드 완료
✅ 1504.png 다운로드 완료
✅ 1506.png 다운로드 완료
✅ 1507.png 다운로드 완료
✅ 1508.png 다운로드 완료
✅ 1509.png 다운로드 완료
✅ 1510.png 다운로드 완료
✅ 1511.png 다운로드 완료
✅ 1512.png 다운로드 완료
✅ 1515.png 다운로드 완료
✅ 1516.png 다운로드 완료
✅ 1517.png 다운로드 완료
✅ 1518.png 다운로드 완료
✅ 1519.png

### 티어 

In [3]:
queue = 'RANKED_SOLO_5x5'
tier = 'CHALLENGER' 
division = 'I'  ## 챌린저, 그마, 마스터는 1,2,3,4 구분 없으므로 I 로 고정
url =f'https://kr.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{division}'

In [15]:
player_list = requests.get(f'https://kr.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{division}', 
                           headers=REQUEST_HEADERS).json()

In [24]:
summoner_ids = [item['summonerId'] for item in player_list]

In [4]:

url = f'https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page={1}'
print(len(requests.get(url, headers=REQUEST_HEADERS).json()))  ## 

205


In [120]:
url = f'https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/{tier}/I?page={1}&api_key={summoner_ids[0]}'

for page in range(1, 20):
    url = f'https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/{tier}/I?page={page}&api_key={summoner_ids[0]}'
    print(len(requests.get(url, headers=REQUEST_HEADERS).json()))  ## 

205
95
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [26]:
tiers = ['CHALLENGER','GRANDMASTER','MASTER']

In [27]:
user_url = 'https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/{}/I?page={}&api_key={}'

In [3]:
import pandas as pd

In [29]:
user_tier = pd.DataFrame()
user_tier

""


In [30]:
user_tier = pd.DataFrame()

for tier in tiers: 
    for page_num in range(1,20):
        # url formatting
        url_now = user_url.format(tier,page_num,api_key)
        
        # request
        reqq =  requests.get(url_now).text
        
        if reqq == '[]':
            pass
        else:
            df = json.loads(reqq)
            user_tier = pd.concat([user_tier, pd.DataFrame(df)], ignore_index=True)

In [18]:
user_tier = user_tier.drop(['leagueId', 'queueType', 'rank', 'summonerId', 'inactive', 'freshBlood', 'hotStreak'], axis = 1)
user_tier

,tier,leaguePoints,wins,losses,veteran
0,CHALLENGER,1755,292,229,True
1,CHALLENGER,1716,180,116,True
2,CHALLENGER,1716,175,118,True
3,CHALLENGER,1695,200,129,True
4,CHALLENGER,1692,159,104,True
...,...,...,...,...,...
4890,MASTER,51,43,20,False
4891,MASTER,51,42,32,False
4892,MASTER,51,34,20,False
4893,MASTER,50,300,318,False


In [31]:
user_tier.groupby('tier').agg(count=('tier', 'count')) ### 다음 할 것: 티어별 통계 본격적 알아보기!

,count
tier,
CHALLENGER,300
GRANDMASTER,700
MASTER,3895


## 각 유저의 챔피언 숙련도

In [8]:
userNickname="세상을 향해 외쳐라"
tagLine="korea"
encodedName = parse.quote(userNickname)
user_url = f"https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{encodedName}/{tagLine}"  ## 해당 api 통해 puuid 얻음

player_id = requests.get(user_url, headers=REQUEST_HEADERS).json()
puuid = player_id['puuid']

champion_url = f"https://kr.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}"
print(puuid)

WgXkRBJDy79YL7GIOSc0nC-WeTSEo8K8neexyTRcSDVxcMIYP1K0NGz3xRpXduWUjvNbaBAheOIX5A


In [33]:
champion_info = requests.get(champion_url, headers = REQUEST_HEADERS).json()

In [34]:
champion_info[0]  ## 유저의 숙련도 가장 높은 챔피언

{'puuid': 'WgXkRBJDy79YL7GIOSc0nC-WeTSEo8K8neexyTRcSDVxcMIYP1K0NGz3xRpXduWUjvNbaBAheOIX5A',
 'championId': 68,
 'championLevel': 24,
 'championPoints': 312040,
 'lastPlayTime': 1732385468000,
 'championPointsSinceLastLevel': 82440,
 'championPointsUntilNextLevel': -71440,
 'markRequiredForNextLevel': 2,
 'tokensEarned': 1,
 'championSeasonMilestone': 5,
 'milestoneGrades': ['A+', 'S-', 'A', 'S-', 'A'],
 'nextSeasonMilestone': {'requireGradeCounts': {'S-': 3},
  'rewardMarks': 1,
  'bonus': True,
  'totalGamesRequires': 3}}

### 매치 데이터

In [3]:
userNickname="hide on bush"
tagLine="kr1"
encodedName = parse.quote(userNickname)
user_url = f"https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{encodedName}/{tagLine}"  ## 해당 api 통해 puuid 얻음

player_id = requests.get(user_url, headers=REQUEST_HEADERS).json()
puuid = player_id['puuid']

match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}"
match_info = requests.get(match_url, headers = REQUEST_HEADERS).json()

In [4]:
puuid

'AIXicczfZtaPXTAWFN3TIe0Bs39QIR3A-_bwkjial05La68y3GsXP2OD88AVcmURZgn3HkxL-R4C7w'

In [5]:
start = 0
count = 20
match_list = requests.get(f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}&api_key={api_key}", headers = REQUEST_HEADERS).json()

In [6]:
match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_list[0]}?api_key={api_key}"

In [7]:
match_info = requests.get(match_url, headers = REQUEST_HEADERS).json()

In [27]:
match_info['info']['participants'][0]['lane']


'TOP'

In [35]:
puuid = 't7IEmGdwLlUXnSAYCfTyoqrqAcX1ixqHInaDrIWVeJ6AdSlJ4TKvPI5jDCsHwrBV5Li4lrnyEbHsoQ'  ## match  에서 추출가능

player = requests.get(f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}", headers=REQUEST_HEADERS).json()
playerInfo = requests.get("https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/"+player['id'], headers = REQUEST_HEADERS).json();
playerInfo[0] ## 매치에서 추출한 정보 활용해 유저 확인 가능

{'leagueId': '04ff61f7-def7-4c92-92b7-6a700fbea8e2',
 'queueType': 'RANKED_FLEX_SR',
 'tier': 'EMERALD',
 'rank': 'I',
 'summonerId': 'jPFTA2WN4EhmVx4n39UYkGF7KXIetFlgZ2C-EcRbUjSHhwk',
 'leaguePoints': 14,
 'wins': 1,
 'losses': 6,
 'veteran': False,
 'inactive': False,
 'freshBlood': False,
 'hotStreak': False}

### 구현하고 싶은 기능은 무엇인가?
#### -> 상위 티어에서의 챔피언 승률  ++ 각 챔피언이 승리했을 때의 평균 시간
#### -> 패치 변화에 따른 승률 변화

In [41]:
match_info['info']['participants'][0]['championId']  ## match 데이터에서 해당 유저 어떤 챔피언 사용했는지 확인 가능

68

In [65]:
# url = f"https://kr.api.riotgames.com/lol/platform/v3/champion-rotations?api_key={api_key}"

In [195]:
champion_constant = requests.get("https://ddragon.leagueoflegends.com/cdn/14.22.1/data/en_US/champion.json")

response = requests.get("https://ddragon.leagueoflegends.com/cdn/14.22.1/data/en_US/champion.json")
data = response.json()

In [196]:
df = pd.DataFrame(data['data']).T
df = df[['id', 'key', 'name']]

In [197]:
df  ## 챔피언 디코딩 사전

,id,key,name
Aatrox,Aatrox,266,Aatrox
Ahri,Ahri,103,Ahri
Akali,Akali,84,Akali
Akshan,Akshan,166,Akshan
Alistar,Alistar,12,Alistar
...,...,...,...
Zeri,Zeri,221,Zeri
Ziggs,Ziggs,115,Ziggs
Zilean,Zilean,26,Zilean
Zoe,Zoe,142,Zoe


In [27]:
user_df = pd.DataFrame()  ## 여기서부터 다시 할 필요 잇음. 유저별로 데이터프레임 만들기

In [7]:
import nest_asyncio
import asyncio
import aiohttp
nest_asyncio.apply()

In [5]:
tier_list= ['CHALLENGER','GRANDMASTER','MASTER']
summoner_ids_list = []
user_df = pd.DataFrame()

for tier in tier_list:
    if tier == 'CHALLENGER':
        page_range = 2
    elif tier == 'GRANDMASTER':
        page_range = 4
    else:
        page_range = 20
    for page in range(1, page_range):
        url = f'https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/{tier}/I?page={page}'
    
        player_list = requests.get(url, headers=REQUEST_HEADERS).json()
        summoner_ids = [{'tier': tier, 'summonerId': item['summonerId']} for item in player_list]
        summoner_ids_list.extend(summoner_ids)

In [14]:
async def fetch_player_data(session, player):
    summoner_id = player['summonerId']
    tier = player['tier']
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"

    while True:
        async with session.get(url) as response:
            if response.status == 200:
                player_info = await response.json()
                return {'tier': tier, 'summonerId': summoner_id, 'puuid': player_info['puuid']}
            elif response.status == 429:
                retry_after = int(response.headers.get('Retry-After', 1))
                await asyncio.sleep(retry_after)
            else:
                print(f"Error fetching PUUID for {summoner_id}: {response.status}")
                return None

# 비동기적으로 모든 소환사 데이터 가져오기
async def main():
    user_data = []
    async with aiohttp.ClientSession(headers=REQUEST_HEADERS) as session:
        tasks = [fetch_player_data(session, player) for player in summoner_ids_list[:1000]]
        for task in asyncio.as_completed(tasks):
            result = await task
            if result:
                user_data.append(result)
    return user_data

# 비동기 루프 실행
start_time = time.time()
user_data = asyncio.run(main())
end_time = time.time()

# 결과를 DataFrame으로 변환
user_df = pd.DataFrame(user_data)

# 확인
print(f"Data fetched in {end_time - start_time:.2f} seconds")
print(user_df.head())
    


Data fetched in 1277.05 seconds
          tier                                         summonerId  \
0   CHALLENGER    HU3gA3OZFyjK_jAv-begvnOG34ji64DacQQTpr-qFnM8FIE   
1  GRANDMASTER    WOwQr-395PHcMaGKHJqovFATfHW-Vp7tb-F9cFyjQfMQ_vY   
2  GRANDMASTER  3k8sn4juioxYVzZ_nH3VS9Q57sHQCYwViY6zqIur846rD1...   
3  GRANDMASTER   L4qQXXVpoxtAJJaT9anBfPM-Z2RLtvl7PXU50y3KDIsSWacF   
4  GRANDMASTER  M8OKeTap-xScCLxYVSsqPmZudXaPkKsPtxCl4vYtWxshO4...   

                                               puuid  
0  ZBvzRh3b0WprDw0AzPk1eZqZMHbk93gDvikLGQsP_rtXfv...  
1  _LJKXSIQQi7TKLunpn3RVqr2-Q6Vj3a2OdT41LLOm_OZNE...  
2  PS6DWafemOytxYyuWqmj7DxEvwO6nElIAbfieHHn3Z4cpy...  
3  cvcVrB39HFvCH33U_3nuTK622wXL__CZbAzpkgpxnD9pUc...  
4  QNlf9GBwnvGrLRY3wwIA1V5eFas8ca0_rN244QaGnJe9nK...  


In [17]:
user_df.head()

,tier,summonerId,puuid
0,CHALLENGER,HU3gA3OZFyjK_jAv-begvnOG34ji64DacQQTpr-qFnM8FIE,ZBvzRh3b0WprDw0AzPk1eZqZMHbk93gDvikLGQsP_rtXfv...
1,GRANDMASTER,WOwQr-395PHcMaGKHJqovFATfHW-Vp7tb-F9cFyjQfMQ_vY,_LJKXSIQQi7TKLunpn3RVqr2-Q6Vj3a2OdT41LLOm_OZNE...
2,GRANDMASTER,3k8sn4juioxYVzZ_nH3VS9Q57sHQCYwViY6zqIur846rD1...,PS6DWafemOytxYyuWqmj7DxEvwO6nElIAbfieHHn3Z4cpy...
3,GRANDMASTER,L4qQXXVpoxtAJJaT9anBfPM-Z2RLtvl7PXU50y3KDIsSWacF,cvcVrB39HFvCH33U_3nuTK622wXL__CZbAzpkgpxnD9pUc...
4,GRANDMASTER,M8OKeTap-xScCLxYVSsqPmZudXaPkKsPtxCl4vYtWxshO4...,QNlf9GBwnvGrLRY3wwIA1V5eFas8ca0_rN244QaGnJe9nK...


In [19]:
def create_database_if_not_exists():
    try:
        connection = pymysql.connect(  ## mysql 연결  단순 작업이므로 connect 사용
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD
        )
        cursor = connection.cursor() ## 데이터 통신 위한 객체 생성 

        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
        print(f"Database '{DB_NAME}' created or already exists.")

        # 연결 종료
        cursor.close()
        connection.close()

    except pymysql.MySQLError as e:
        print(f"Error creating database: {e}")

def get_engine():
    create_database_if_not_exists()  # 데이터베이스가 없으면 생성
    engine = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')  ## 데이터베이스 연결  하이 레벨의 연결 제공
    return engine

if __name__ == "__main__":
    API_KEY = 'RGAPI-cf6451c4-7371-4d57-8ab1-c890743fe164'
    REQUEST_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": API_KEY
    }

    DB_USER = 'root'
    DB_PASSWORD = '!!good8236'
    DB_HOST = 'localhost'
    DB_NAME = 'high_user_puuid'

    # summoner_data = get_summoner_data(summoner_name, tagLine)
    engine = get_engine()

    df = user_df

    if not df.empty:
        df.to_sql('user_tier', con=engine, if_exists='replace', index=False)

Database 'high_user_puuid' created or already exists.


In [4]:
match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}"
match_info = requests.get(match_url, headers = REQUEST_HEADERS).json()

start = 0
count = 20
match_list = requests.get(f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}&api_key={api_key}", headers = REQUEST_HEADERS).json()


NameError: name 'puuid' is not defined

In [54]:
match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_list[0]}?api_key={api_key}"
match_info = requests.get(match_url, headers = REQUEST_HEADERS).json()

In [61]:
match_info['info']['participants'][0]['championId']

24

In [66]:
df[df['key'] == '24']  ## 해당 유저

,id,key,name
Jax,Jax,24,Jax


In [ ]:
## 챌린저 유저들의 매치데이터에서 각 챔피언의 승률 추출해보기

In [1]:
import nest_asyncio
import asyncio
import aiohttp

nest_asyncio.apply()

In [122]:
async def fetch_puuid_with_retry(session, summoner_id, max_retries=5):
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"
    retries = 0

    while retries < max_retries:
        async with session.get(url, headers=REQUEST_HEADERS) as response:
            if response.status == 200:  # 성공적인 요청
                data = await response.json()
                return data.get('puuid')
            elif response.status == 429:  # Rate Limit 초과
                retry_after = int(response.headers.get('Retry-After', 1))  # 대기 시간
                print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
                await asyncio.sleep(retry_after)
                retries += 1
            else:
                print(f"Error: {response.status}")
                retries += 1
    print(f"Failed to fetch puuid for summoner_id: {summoner_id}")
    return None  # 요청 실패 시 None 반환

async def fetch_all_puuids(summoner_ids, max_retries=5):
    async with aiohttp.ClientSession() as session:
        puuids = []
        for i in range(0, len(summoner_ids), 20):  # 20개씩 묶어서 요청
            batch = summoner_ids[i:i+20]
            tasks = [fetch_puuid_with_retry(session, summoner_id, max_retries) for summoner_id in batch]
            batch_results = await asyncio.gather(*tasks)
            puuids.extend(batch_results)
            await asyncio.sleep(1)  # 요청 간 대기 시간 추가 (Rate Limit 방지)
        return puuids

async def retry_missing_puuids(summoner_ids, existing_puuids):
    missing_ids = [id for id, puuid in zip(summoner_ids, existing_puuids) if puuid is None]
    if missing_ids:
        print(f"Retrying for {len(missing_ids)} missing summoner IDs...")
        retry_results = await fetch_all_puuids(missing_ids)
        return retry_results
    return []

async def main(challenger_summoner_ids):
    # Step 1: First attempt to fetch all PUUIDs
    puuids = await fetch_all_puuids(challenger_summoner_ids)
    # Step 2: Retry for missing PUUIDs
    retries = await retry_missing_puuids(challenger_summoner_ids, puuids)
    # Step 3: Replace missing values with retries
    for i, puuid in enumerate(puuids):
        if puuid is None and i < len(retries):
            puuids[i] = retries.pop(0)
    # Filter final PUUID list (remove None)
    puuids = [p for p in puuids if p is not None]
    return puuids


puuids = asyncio.run(main(challenger_summoner_ids))

print(f"Total PUUIDs fetched: {len(puuids)}")

Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying after 115 seconds...
Rate limit exceeded. Retrying a

In [124]:
puuids = [puuid for puuid in puuids if puuid is not None]
len(puuids)

200

In [ ]:
## 한명의 유저에 대해서 20개 match data 뽑고 각 챔피언 승률 보기

In [139]:
match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuids[0]}/ids?start={start}&count={count}"
match_ids = requests.get(match_url, headers=REQUEST_HEADERS).json()  ## 총 20개의 match data

In [141]:
def get_match_info(match_ids, api_key=api_key):
    match_info_list = []
    for match_id in match_ids:
        match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
        response = requests.get(match_url, headers=REQUEST_HEADERS)
        match_info_list.append(response.json())
    return match_info_list

In [142]:
match_info = get_match_info(match_ids, api_key=api_key)

In [146]:
champion_list = []
for match in match_info:
    champion = match['info']['participants'][0]['championId']
    champion_list.append(champion)

In [152]:
one_user_df = pd.DataFrame(columns = ['champion', 'winrate'])

In [30]:
response = requests.get("https://ddragon.leagueoflegends.com/cdn/14.22.1/data/en_US/champion.json")
data = response.json()
df = pd.DataFrame(data['data']).T
df = df[['id', 'key', 'name']]

df['key'] = df['key'].astype('int64')

In [198]:
import pandas as pd
from collections import defaultdict

champion_stats = defaultdict(lambda: {'wins': 0, 'games': 0})

for match in match_info:
    participants = match['info']['participants']
    
    for participant in participants:
        champion = participant['championId']
        win = participant['win']
        
        # 챔피언에 대해 승리 여부와 게임 수를 기록
        champion_stats[champion]['games'] += 1
        if win:
            champion_stats[champion]['wins'] += 1

# 데이터를 DataFrame으로 변환
champion_data = []
for champion, stats in champion_stats.items():
    win_rate = stats['wins'] / stats['games'] * 100  # 승률을 백분율로 계산
    champion_data.append({
        'key': champion,
        'Games Played': stats['games'],
        'Wins': stats['wins'],
        'Win Rate (%)': round(win_rate, 2)
    })

# DataFrame 생성
df_champion_stats = pd.DataFrame(champion_data)

df['key'] = df['key'].astype('int64')
champion_map = df.set_index('key')['id'].to_dict()  ## 이 코드 수행해보기
df_champion_stats['champion'] = df_champion_stats['key'].map(champion_map)


df_champion_stats

    key  Games Played  Wins  Win Rate (%)
0    24             6     1         16.67
1   200             2     1         50.00
2    55             1     1        100.00
3    42             7     6         85.71
4   111             4     3         75.00
..  ...           ...   ...           ...
77  526             1     1        100.00
78   39             1     0          0.00
79   18             1     0          0.00
80   54             1     0          0.00
81   91             1     0          0.00

[82 rows x 4 columns]


In [203]:
# df['key'] = df['key'].astype('int64')

In [204]:
# champion_map = df.set_index('key')['id'].to_dict()  ## 이 코드 수행해보기
# df_champion_stats['champion_name'] = df_champion_stats['key'].map(champion_map)

In [205]:
# df_champion_stats

,key,Games Played,Wins,Win Rate (%),champion_name
0,24,6,1,16.67,Jax
1,200,2,1,50.00,Belveth
2,55,1,1,100.00,Katarina
3,42,7,6,85.71,Corki
4,111,4,3,75.00,Nautilus
...,...,...,...,...,...
77,526,1,1,100.00,Rell
78,39,1,0,0.00,Irelia
79,18,1,0,0.00,Tristana
80,54,1,0,0.00,Malphite


In [187]:
# df['key'] = df['key'].astype('int64')

In [188]:
# merged_df = pd.merge(df_champion_stats, df, on='key', how='left') 
# merged_df  ## 한 유저의 20판 게임에서의 챔피언 승률

,Champion_x,Games Played,Wins,Win Rate (%),Champion Name,Champ,key,id,name,Champion_y
0,24,6,1,16.67,NaN,NaN,24,Jax,Jax,Jax
1,200,2,1,50.00,NaN,NaN,200,Belveth,Bel'Veth,Belveth
2,55,1,1,100.00,NaN,NaN,55,Katarina,Katarina,Katarina
3,42,7,6,85.71,NaN,NaN,42,Corki,Corki,Corki
4,111,4,3,75.00,NaN,NaN,111,Nautilus,Nautilus,Nautilus
...,...,...,...,...,...,...,...,...,...,...
77,526,1,1,100.00,NaN,NaN,526,Rell,Rell,Rell
78,39,1,0,0.00,NaN,NaN,39,Irelia,Irelia,Irelia
79,18,1,0,0.00,NaN,NaN,18,Tristana,Tristana,Tristana
80,54,1,0,0.00,NaN,NaN,54,Malphite,Malphite,Malphite


In [193]:
df = pd.DataFrame({
    'key': [101, 102, 103],  # 챔피언 코드
    'id': ['Annie', 'Olaf', 'Ryze']  # 실제 챔피언 이름
})

# 예시 champion_stat - 챔피언 코드로 되어 있는 통계 DataFrame
champion_stat = pd.DataFrame({
    'champion_code': [101, 102, 103, 101, 103],  # 챔피언 코드
    'games_played': [10, 15, 20, 5, 12],  # 게임 수
    'wins': [6, 9, 15, 3, 10]  # 승리 수
})

# df의 'key' 컬럼을 'id'로 매핑한 시리즈를 생성
champion_map = df.set_index('key')['id'].to_dict()

# champion_stat의 'champion_code' 컬럼을 'map'으로 매핑
champion_stat['champion_name'] = champion_stat['champion_code'].map(champion_map)

# 결과 출력
champion_stat

,champion_code,games_played,wins,champion_name
0,101,10,6,Annie
1,102,15,9,Olaf
2,103,20,15,Ryze
3,101,5,3,Annie
4,103,12,10,Ryze


In [120]:
def fetch_match_list(puuid, start=0, count=20):

    match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}"
    response = requests.get(match_url, headers=REQUEST_HEADERS)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match list for {puuid}: {response.status_code}")
        return None

# 실행
match_results = []
for puuid in puuids:
    matches = fetch_match_list(puuid)
    match_results.append(matches)

Error fetching match list for zuVV6-Y48We8SVyI5sivA76_JjFkrlUV2jqvRnzOzgxmgR-HOpveCNHxhBOCXNs-VxjX0AyhBM5PYQ: 429
Error fetching match list for rTYys5GlRuKbcLICjiU8FBpkW7Yue6d2xdilv84hLuT2mPvW8QiEXYTaP9-JAiAfcnOI1DIkNaXbyw: 429
Error fetching match list for QXoUR54oO1cW-vhDF7bipDe8WCq9vz6tZzrqOD8w5K-fLZSVS5Dkt5KbqyVAmpboeS2V7SOOZm2Uig: 429
Error fetching match list for bRkh2bWr30eVc5f1fA4v3uk-lQLOzmr_Lv_Ue7mzYojGy6o4R64bvhye-In_SUXskdcfWr8dPre8IQ: 429
Error fetching match list for UjDlx-YdBSlxl6UNBRJtn1DfW8ZKU_euEoixWz3wcxsSn7cSPRuAqMFQVsbejJG_hSaOf80VKlxnaA: 429
Error fetching match list for DTUeZBhciRwk2FdS6HLP_0RCs7qKICUjGyxbSPDb6An9gY0O_JpUN7c_pTkidb1fFwldCQztHftGQw: 429
Error fetching match list for emivnZnBkJirW-XUedJ_W5K_IG3kun_4sU7n2aJXwjLNaUlwGJvJRLtvDxl8Ybpq32g63GzLhOldEA: 429
Error fetching match list for oMrKYi1gmX4absUAvjo6eCR1tzpoKHeDsopNx81KvdGG1m0zD5HWjBKVvp-rUhUsXD0g6cXzd6rakg: 429
Error fetching match list for Xok063Y6KbghT8wl75QnmJG3EGzZdAyqgo-K5E7MzF_h4aMegVoY-NJ2A9

Error fetching match list for Gn5dbO6VbbR_GDomrwY0UbH_rLD696g0xCfH019sS6GuIApYLQOU3cqB7ZpWJi2XYVayrbms2qm-4g: 429
Error fetching match list for vzr28eWCxfZ3GgXYgan_UZmZLNuEuM2zdQwXFyj-m6nkVTnzpA1K2o3D1HJv5t2OEeQI0dMIdezxHA: 429
Error fetching match list for dZrsuMsFm9n7-ON8xsiLWKq6dzLswlTmwe994yjF6-arW7PkjIlgWCBUwo0KZahJSK_1TCh61PaV8w: 429
Error fetching match list for OZSRekdAKNdCvEdUoii89eDWvg-Dpu0FyijaK-BvaqjWZ1QLqwfneGtDJxEltaJjT-UKUgCiQAqx8Q: 429
Error fetching match list for E_jyMvCablX1Z1dlS3orYxX08C2Bv0YhFZtXvGWkmRCX1CRy5uH8dKjL__w2GkjJW2LcChFrv92Qug: 429
Error fetching match list for 07FvThxdHyDkSqJk180VzNtvtMw5ROqEXabGlrmJUbdu8iFjZo5OB1mwSWu9vcgnIxy349XhfQqN6Q: 429
Error fetching match list for IMy3-_SG5qBlUW6evdRO-zDuZ-0A28GELom4fZbvwnJkwmpF-eItL8c6k3_fgihkwMvJ4mzGU8PbbA: 429
Error fetching match list for YA2hO8QW2CP2Cc0f43Fawb914zGQZH4m8LuXpMVPuF0eCE6IfgsA-V5mX9WQUL4e8F6SlOCjkp0hSg: 429
Error fetching match list for GjjoEjs6xvlV0aAaBchCbCL-1iaWTDrrg98nkoNxeN2hNqoZgMtXj_wHg4

In [37]:
api_key =  'RGAPI-d04f6e5a-d60d-422d-bdda-4f45905eb951' # 본인의 API KEY 입력
REQUEST_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

In [42]:
tiers = ["CHALLENGER"]

user_tier = pd.DataFrame()
print('상위 티어 데이터 불러오는 중')
page_range = 1
for tier in tiers: 
    for page in range(1, 2):
        user_url = f'https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/{tier}/I?page={page}&api_key={api_key}'
        url_now = user_url.format(tier,page, api_key)
        reqq =  requests.get(url_now).text
        if reqq == '[]':
            pass
        else:
            df = json.loads(reqq)
            user_tier = pd.concat([user_tier, pd.DataFrame(df)], ignore_index=True)
        break
    break
user_tier_df = user_tier.drop(['leagueId', 'queueType', 'rank', 'inactive', 'freshBlood', 'hotStreak'], axis = 1)
print('PUUID 불러오는 중')
puuids = []
for summonerId in user_tier_df['summonerId']:
    user_puuids_url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}"
    response = requests.get(user_puuids_url, headers=REQUEST_HEADERS)
    
    if response.status_code == 200:  
        puuid = response.json().get('puuid')
        if puuid:  
            puuids.append(puuid)
        else:
            print(f"PUUID가 없는 응답: {response.json()}")
            puuids.append(None)  
            
    elif response.status_code == 429:  # Rate Limit 초과
        # retry_after = int(response.headers.get("Retry-After", 10))  # Retry-After 헤더 값 확인 (없으면 10초)
        print(f"Rate Limit 초과...")
        # time.sleep(retry_after)  # 대기 후 재시도

상위 티어 데이터 불러오는 중
PUUID 불러오는 중
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
Rate Limit 초과...
R

In [43]:
user_tier_df

,tier,summonerId,leaguePoints,wins,losses,veteran
0,CHALLENGER,9IysF5dRiW9Ai2LuiUzGsx7Hd7GooeyYBqFoxJJXsugzoXs,1653,121,84,True
1,CHALLENGER,5gVq2gWAEWub8El7yo8QCZczEnQfKxnBdHAB0GX5q_6O_g,1594,117,78,True
2,CHALLENGER,U0ahCGbkrdzNp94wGkVX1ppix2a_YjRm5FVpKb7A4Yr6YpR4,1577,185,136,True
3,CHALLENGER,cDr0VNE09w94lvCdb2r2cDlEmya3WTy9UdqJh6ze8CBaQI...,1543,196,164,True
4,CHALLENGER,tLR-9bFFGkj3qWQGARhCt8wYdio7ganHzpwDKXYWkUgCrIc,1503,256,208,True
...,...,...,...,...,...,...
200,CHALLENGER,PB_PJsdlHYM1p-7RpO2r57tRcSOm0EjwTiZwnOVxFradM3NM,863,76,61,False
201,CHALLENGER,G86AOXPMmyBbE_HYPz6znt0NcWgmrFXvg67MU3jv7LT8uA...,862,81,56,False
202,CHALLENGER,RtPs87kx1WpKSVq9E600m67GscDrK4gcCNvW9ku0Ew2onm8,862,73,61,False
203,CHALLENGER,jDIdZXfy8EkDACl7cX1XPWbJWaVRNwBKM0auewvHFqGW_1E,861,64,52,False


In [49]:
from collections import defaultdict

In [69]:
len(puuids)

147

In [ ]:
puuids = []
for summonerId in user_tier_df['summonerId']:
    user_puuids_url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}"
    response = requests.get(user_puuids_url, headers=REQUEST_HEADERS)
    
    if response.status_code == 200:  
        puuid = response.json().get('puuid')
        if puuid:  
            puuids.append(puuid)
        else:
            print(f"PUUID가 없는 응답: {response.json()}")
            puuids.append(None)  
            
    elif response.status_code == 429:  # Rate Limit 초과
        # retry_after = int(response.headers.get("Retry-After", 10))  # Retry-After 헤더 값 확인 (없으면 10초)
        print(f"Rate Limit 초과. ")
        # time.sleep(retry_after)  # 대기 후 재시도

In [6]:
import requests
import os

# 챔피언 리스트 저장
champion = []

# JSON 파일 가져오기
datas = requests.get('https://ddragon.leagueoflegends.com/cdn/15.2.1/data/en_US/champion.json')
datas = datas.json()

for data in datas["data"]:
    champion.append(data)

# 저장할 폴더 경로 설정
save_folder = r"C:\Users\JiWoongChoi\pythonproject\riotproject\riotproject\static\images"

# 폴더 없으면 생성
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

for champ in champion:
    
    save_path = os.path.join(save_folder, champ + ".png")

    if os.path.exists(save_path):
        print(f"⏩ {champ}.png 이미 존재하여 건너뜀")
        continue

    image_url = f"https://ddragon.leagueoflegends.com/cdn/15.2.1/img/champion/{champ}.png"

    download_image = requests.get(image_url)

    if download_image.status_code == 200:  # 다운로드 성공 여부 확인
        with open(save_path, 'wb') as file:
            file.write(download_image.content)
        print(f"✅ {champ}.png 다운로드 완료")
    else:
        print(f"❌ {champ}.png 다운로드 실패 (HTTP {download_image.status_code})")

⏩ Aatrox.png 이미 존재하여 건너뜀
⏩ Ahri.png 이미 존재하여 건너뜀
⏩ Akali.png 이미 존재하여 건너뜀
⏩ Akshan.png 이미 존재하여 건너뜀
⏩ Alistar.png 이미 존재하여 건너뜀
✅ Ambessa.png 다운로드 완료
⏩ Amumu.png 이미 존재하여 건너뜀
⏩ Anivia.png 이미 존재하여 건너뜀
⏩ Annie.png 이미 존재하여 건너뜀
⏩ Aphelios.png 이미 존재하여 건너뜀
⏩ Ashe.png 이미 존재하여 건너뜀
⏩ AurelionSol.png 이미 존재하여 건너뜀
✅ Aurora.png 다운로드 완료
⏩ Azir.png 이미 존재하여 건너뜀
⏩ Bard.png 이미 존재하여 건너뜀
✅ Belveth.png 다운로드 완료
⏩ Blitzcrank.png 이미 존재하여 건너뜀
⏩ Brand.png 이미 존재하여 건너뜀
⏩ Braum.png 이미 존재하여 건너뜀
✅ Briar.png 다운로드 완료
⏩ Caitlyn.png 이미 존재하여 건너뜀
⏩ Camille.png 이미 존재하여 건너뜀
⏩ Cassiopeia.png 이미 존재하여 건너뜀
⏩ Chogath.png 이미 존재하여 건너뜀
⏩ Corki.png 이미 존재하여 건너뜀
⏩ Darius.png 이미 존재하여 건너뜀
⏩ Diana.png 이미 존재하여 건너뜀
⏩ Draven.png 이미 존재하여 건너뜀
⏩ DrMundo.png 이미 존재하여 건너뜀
⏩ Ekko.png 이미 존재하여 건너뜀
⏩ Elise.png 이미 존재하여 건너뜀
⏩ Evelynn.png 이미 존재하여 건너뜀
⏩ Ezreal.png 이미 존재하여 건너뜀
⏩ Fiddlesticks.png 이미 존재하여 건너뜀
⏩ Fiora.png 이미 존재하여 건너뜀
⏩ Fizz.png 이미 존재하여 건너뜀
⏩ Galio.png 이미 존재하여 건너뜀
⏩ Gangplank.png 이미 존재하여 건너뜀
⏩ Garen.png 이미 존재하여 건너뜀
⏩ Gnar.png 이미 존재하여 건너뜀
⏩ Gragas.p

In [ ]:
## 밑의 코드들은 위의 코드와 결합하여 실험 가능능

In [99]:
puuid = puuids[0]
start= 0
count =10
match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}"
match_ids = requests.get(match_url, headers=REQUEST_HEADERS).json()

In [100]:
match_id = match_ids[0]

match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
response = requests.get(match_url, headers=REQUEST_HEADERS)

In [127]:
alist = []
for puuid in puuids:
    start= 0
    count =10
    match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}"
    match_ids = requests.get(match_url, headers=REQUEST_HEADERS).json()

    for match_id in match_ids:
        match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
        response = requests.get(match_url, headers=REQUEST_HEADERS)
        if response.status_code != 200:
            print(f"Error fetching match data for match_id: {match_id}, Status: {response.status_code}, Message: {response.text}")
            continue
        else:
            mode = response.json()['info']['gameMode']
        if mode == 'CLASSIC':
            continue
        else:
            print(mode)
        alist.append(match_id)
            

ARAM
ARAM
ARAM
ARAM
URF
ARAM
ARAM
ARAM
Error fetching match data for match_id: KR_7503433334, Status: 429, Message: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match data for match_id: KR_7503370881, Status: 429, Message: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match data for match_id: KR_7501094555, Status: 429, Message: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match data for match_id: KR_7501033292, Status: 429, Message: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match data for match_id: KR_7500989426, Status: 429, Message: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match data for match_id: KR_7500895125, Status: 429, Message: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error fetching match data for match_id: status, Status: 429, Message: {"status":{"message":"Rate limit exceeded","status_

KeyboardInterrupt: 

In [70]:
start= 0 
count = 5

top_champion_stats = defaultdict(lambda: {'wins': 0, 'games': 0})
jg_champion_stats = defaultdict(lambda: {'wins': 0, 'games': 0})
mid_champion_stats = defaultdict(lambda: {'wins': 0, 'games': 0})
bot_champion_stats = defaultdict(lambda: {'wins': 0, 'games': 0})

for puuid in puuids:
    if puuid == None:
        continue
    match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}"
    match_ids = requests.get(match_url, headers=REQUEST_HEADERS).json()

    match_info_list = []
    for match_id in match_ids:
        match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
        response = requests.get(match_url, headers=REQUEST_HEADERS)
        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code} for match ID {match_id}")
            print(f"Response: {response.text}")  ## rate limit 주로 발생
            continue
        else:
            match_info_list.append(response.json())


    for match in match_info_list:
        participants = match['info']['participants']
        
        for participant in participants:
            champion = participant['championId']
            lane = participant['lane']
            win = participant['win']
            
            if lane == 'TOP':
                top_champion_stats[champion]['games'] += 1
                if win:
                    top_champion_stats[champion]['wins'] += 1

            elif lane == 'JUNGLE':
                jg_champion_stats[champion]['games'] += 1
                if win:
                    jg_champion_stats[champion]['wins'] += 1

            elif lane == 'MIDDLE':
                mid_champion_stats[champion]['games'] += 1
                if win:
                    mid_champion_stats[champion]['wins'] += 1

            elif lane == 'BOTTOM':
                bot_champion_stats[champion]['games'] += 1
                if win:
                    bot_champion_stats[champion]['wins'] += 1

top_champion_data = []
jg_champion_data = []
mid_champion_data = []
bot_champion_data = []

for champion, stats in top_champion_stats.items():
    win_rate = stats['wins'] / stats['games'] * 100  # 승률을 백분율로 계산
    top_champion_data.append({
        'key': champion,
        'Games_Played': stats['games'],
        'Wins': stats['wins'],
        'Win_Rate': round(win_rate, 2)
    })
for champion, stats in jg_champion_stats.items():
    win_rate = stats['wins'] / stats['games'] * 100  # 승률을 백분율로 계산
    jg_champion_data.append({
        'key': champion,
        'Games_Played': stats['games'],
        'Wins': stats['wins'],
        'Win_Rate': round(win_rate, 2)
    })

for champion, stats in mid_champion_stats.items():
    win_rate = stats['wins'] / stats['games'] * 100  # 승률을 백분율로 계산
    mid_champion_data.append({
        'key': champion,
        'Games_Played': stats['games'],
        'Wins': stats['wins'],
        'Win_Rate': round(win_rate, 2)
    })

for champion, stats in bot_champion_stats.items():
    win_rate = stats['wins'] / stats['games'] * 100  # 승률을 백분율로 계산
    bot_champion_data.append({
        'key': champion,
        'Games_Played': stats['games'],
        'Wins': stats['wins'],
        'Win_Rate': round(win_rate, 2)
    })

top_df_champion_stats = pd.DataFrame(top_champion_data)
jg_df_champion_stats = pd.DataFrame(jg_champion_data)
mid_df_champion_stats = pd.DataFrame(mid_champion_data)
bot_df_champion_stats = pd.DataFrame(bot_champion_data)

Error: Received status code 429 for match ID KR_7503525348
Response: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error: Received status code 429 for match ID KR_7503433334
Response: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error: Received status code 429 for match ID status
Response: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error: Received status code 429 for match ID status
Response: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error: Received status code 429 for match ID status
Response: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error: Received status code 429 for match ID status
Response: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error: Received status code 429 for match ID status
Response: {"status":{"message":"Rate limit exceeded","status_code":429}}
Error: Received status code 429 for match ID status
Response: {"status":{"message":"Rate limit exceeded","statu

In [74]:
jg_df_champion_stats

,key,Games_Played,Wins,Win_Rate
0,104,10,2,20.00
1,876,10,6,60.00
2,120,5,0,0.00
3,234,40,10,25.00
4,76,22,8,36.36
5,235,1,1,100.00
6,68,2,0,0.00
7,72,28,17,60.71
8,64,24,15,62.50
9,113,3,3,100.00
